### 实战 Kaggle 比赛：预测房价

In [1]:
# 实现几个函数来方便下载数据

# 导入必要的库
import hashlib
import os
import tarfile
import zipfile
import requests

# 定义数据存储信息
DATA_HUB = dict()  # 数据源字典，存储文件名到下载URL和SHA-1哈希值的映射
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'  # 基础数据下载URL前缀


def download(name, cache_dir=os.path.join('..', 'data')):
    """下载一个DATA_HUB中的文件，返回本地文件名"""

    # 检查文件名是否存在于数据源字典中
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}"

    # 获取文件的下载URL和SHA-1哈希值
    url, sha1_hash = DATA_HUB[name]

    # 确保缓存目录存在，不存在则创建
    os.makedirs(cache_dir, exist_ok=True)

    # 生成本地文件路径
    fname = os.path.join(cache_dir, url.split('/')[-1])

    # 如果本地文件已存在
    if os.path.exists(fname):
        sha1 = hashlib.sha1()  # 创建SHA-1哈希对象
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)  # 逐块读取文件内容
                if not data:
                    break
                sha1.update(data)  # 更新哈希值
        # 比较计算得到的哈希值与预定义哈希值是否相等
        if sha1.hexdigest() == sha1_hash:
            return fname  # 命中缓存，返回文件路径

    # 如果文件不存在或哈希值不匹配，进行下载
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)  # 发起HTTP GET请求
    with open(fname, 'wb') as f:
        f.write(r.content)  # 将下载内容写入文件
    return fname  # 返回下载的文件路径

In [2]:
# 使用pandas读入并处理数据
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

DATA_HUB['kaggle_house_train'] = (DATA_URL + 'kaggle_house_pred_train.csv',
                                  '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (DATA_URL + 'kaggle_house_pred_test.csv',
                                 'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [3]:
# 前两个和最后两个特征，以及相应标签（房价）
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [4]:
# 在每个样本中，第一个特征是 ID，我们将其从数据集中删除
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

In [5]:
# 将所有缺失的值替换为相应特征的平均值。通过将特征重新缩放到零均值和单位方差来标准化数据

# 获取所有数值型特征的索引
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index

# 标准化数值型特征
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 上述操作使用匿名函数对每个特征列进行标准化，将值减去均值并除以标准差

# 填充数值型特征中的缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# 上述操作将数值型特征中的缺失值用0来填充

In [6]:
# 处理离散值(字符串)。我们用一次独热编码替换他们
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [7]:
# 从pandas格式中提取NumPy格式，并将其转换为张量表示
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values,
                              dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values,
                             dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1),
                            dtype=torch.float32)

In [8]:
# 训练
loss = nn.MSELoss()
in_features = train_features.shape[1]


def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

**我们更关心相对误差$\frac{y - \hat{y}}{y}$**，解决这个问题的一种方法是用价格**预测的对数**来衡量差异

> **防止对于价格更高的房子，给予了更多的权重。所以算一个百分比，由于有除法，所以用一下log**

当我们在进行回归任务时，我们通常希望我们的预测结果能够与真实的标签尽可能接近。但在现实情况下，由于各种因素，预测值和真实标签之间的差异可能会比较大。有时候，数据中可能还包含一些异常值，这些异常值的出现可能会导致预测误差的不稳定性。

**对数变换**是一种常用的技术，可以帮助我们在处理这种情况时更好地进行分析和建模。下面解释一下**为什么对数变换能够有助于减小异常值的影响，以及如何使得预测值和真实标签之间的差异更具线性性质**：

1. **减小异常值的影响**：
   异常值指的是与其他数据点明显不同的数据点。在回归任务中，如果数据中存在一些异常值，它们可能会影响模型的训练和预测结果，导致模型偏离正常数据的表现。对数变换可以有效地缩小异常值的影响，因为对数变换后，数据中较大的值会被拉近，使其与其他数据更接近。

2. **线性性质**：
   在某些情况下，对数变换可以使数据更加接近线性关系。线性关系对于建立预测模型非常有利，因为很多机器学习算法都假设数据是线性可分的。当预测值和真实标签之间的关系更加线性时，我们可以更好地应用线性模型来拟合数据，从而提高模型的性能。

**总之，通过将预测值和真实标签进行对数变换，我们可以在处理异常值时更加稳健，同时也可以使预测值和真实标签之间的差异更趋向于线性关系**，有助于更好地训练和评估预测模型。这对于建立高质量的回归模型非常重要。

In [9]:
def log_rmse(net, features, labels):
    """
    计算对数均方根误差（log RMSE）作为模型性能指标。
    
    参数：
        net (nn.Module): 神经网络模型，用于预测特征 features 的标签
        features (torch.Tensor): 输入特征数据
        labels (torch.Tensor): 真实标签数据
    
    返回：
        float: 对数均方根误差的标量值
    """
    # 对预测值进行剪裁，确保其不会小于1或大于正无穷大
    clipped_preds = torch.clamp(net(features), 1, float('inf'))

    # 计算对数均方根误差（log RMSE）
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))

    return rmse.item()


# 注解：
# net: 神经网络模型，用于预测特征 features 的标签
# features: 输入特征数据
# labels: 真实标签数据
# torch.clamp(input, min, max): 对输入的张量进行剪裁，限制其取值范围在 min 和 max 之间
# torch.log(input): 计算输入张量的自然对数
# torch.sqrt(input): 计算输入张量的平方根
# loss: 损失函数，用于计算预测值与真实标签之间的差异
# rmse.item(): 将计算得到的对数均方根误差的张量值转换为Python标量值

In [10]:
# 我们的训练函数借助Adam优化器
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    """
    训练神经网络模型，并评估性能。

    参数：
        net (nn.Module): 要训练的神经网络模型
        train_features (torch.Tensor): 训练集特征数据
        train_labels (torch.Tensor): 训练集标签数据
        test_features (torch.Tensor): 测试集特征数据
        test_labels (torch.Tensor): 测试集标签数据
        num_epochs (int): 训练周期数
        learning_rate (float): 学习率
        weight_decay (float): 权重衰减（L2正则化）系数
        batch_size (int): 批量大小
    
    返回：
        train_ls (list): 每个训练周期结束后的训练集对数均方根误差列表
        test_ls (list): 每个训练周期结束后的测试集对数均方根误差列表
    """
    train_ls, test_ls = [], []  # 初始化列表以存储训练和测试集的对数均方根误差

    # 创建数据迭代器，将训练数据划分为批次
    train_iter = d2l.load_array((train_features, train_labels), batch_size)

    # 创建 Adam 优化器，用于参数更新
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr=learning_rate,
                                 weight_decay=weight_decay)

    # 在每个训练周期内进行训练和评估
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()  # 清零梯度
            l = loss(net(X), y)  # 计算损失
            l.backward()  # 反向传播计算梯度
            optimizer.step()  # 使用优化器更新参数

        # 计算并记录训练集的对数均方根误差
        train_ls.append(log_rmse(net, train_features, train_labels))

        # 如果提供了测试集标签，则计算并记录测试集的对数均方根误差
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))

    return train_ls, test_ls


# 注解：
# net: 要训练的神经网络模型
# train_features: 训练集特征数据
# train_labels: 训练集标签数据
# test_features: 测试集特征数据
# test_labels: 测试集标签数据
# num_epochs: 训练周期数
# learning_rate: 学习率
# weight_decay: 权重衰减（L2正则化）系数
# batch_size: 批量大小
# d2l.load_array(data, batch_size): 创建数据迭代器，划分数据为批次
# torch.optim.Adam(params, lr, weight_decay): 创建Adam优化器
# optimizer.zero_grad(): 清零梯度
# l.backward(): 反向传播计算梯度
# optimizer.step(): 使用优化器更新参数
# log_rmse(net, features, labels): 计算对数均方根误差
# train_ls: 每个训练周期结束后的训练集对数均方根误差列表
# test_ls: 每个训练周期结束后的测试集对数均方根误差列表

In [11]:
# K 折交叉验证
def get_k_fold_data(k, i, X, y):
    """
    获取 k 折交叉验证中的训练集和验证集数据。
    
    参数：
        k (int): 折数
        i (int): 当前折索引
        X (torch.Tensor): 特征数据
        y (torch.Tensor): 标签数据
    
    返回：
        X_train (torch.Tensor): 训练集特征数据
        y_train (torch.Tensor): 训练集标签数据
        X_valid (torch.Tensor): 验证集特征数据
        y_valid (torch.Tensor): 验证集标签数据
    """
    assert k > 1  # 确保折数大于1

    fold_size = X.shape[0] // k  # 每折的数据大小
    X_train, y_train = None, None  # 初始化训练集数据

    # 迭代折数，生成训练集和验证集数据
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  # 当前折的索引范围
        X_part, y_part = X[idx, :], y[idx]  # 当前折的数据

        if j == i:  # 当前折是验证集的折
            X_valid, y_valid = X_part, y_part
        elif X_train is None:  # 训练集的第一个折
            X_train, y_train = X_part, y_part
        else:  # 其他训练集折
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)

    return X_train, y_train, X_valid, y_valid


# 注解：
# k: 折数
# i: 当前折索引
# X: 特征数据
# y: 标签数据
# assert k > 1: 确保折数大于1
# fold_size: 每折的数据大小
# X_train: 训练集特征数据
# y_train: 训练集标签数据
# X_valid: 验证集特征数据
# y_valid: 验证集标签数据
# slice(start, stop): 创建一个切片对象，用于获取数据索引范围
# X[idx, :]: 使用切片索引获取数据的特征部分
# y[idx]: 使用切片索引获取数据的标签部分
# torch.cat(tensors, dim): 沿给定维度拼接张量

In [12]:
# 返回训练和验证误差的平均值
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay, batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        # 获取第i折的训练和验证数据
        data = get_k_fold_data(k, i, X_train, y_train)
        # 创建模型
        net = get_net()
        # 训练并返回训练和验证误差